In [34]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import os
from matplotlib import pyplot as plt
from PIL import Image
import re
import tensorflow as tf
import keras.losses

# Define your model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(153, 153, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


def iou(y_true, y_pred):
    intersection = tf.reduce_sum(tf.multiply(y_true, y_pred))
    union = tf.reduce_sum(tf.subtract(tf.add(y_true, y_pred), tf.multiply(y_true, y_pred)))
    iou_score = tf.divide(intersection, union)
    return iou_score

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[iou])

# Load and preprocess your data
# Assuming you have images and corresponding masks in separate directories
# You will need to modify this part based on your data directory structure
image_dir = '../cse-seminar/small_images'
mask_dir = '../cse-seminar/labeled_data'
image_files = []  # List of image filenames
mask_files = []  # List of mask filenames

labeled_data  = []
mask = []

for filename in os.listdir(image_dir):
    # Check if the file has a common image extension (e.g., jpg, png, etc.)
    if filename.lower().endswith('.png'):
        # Construct the full file path by joining the folder path and filename
        file_path =  filename
        # Append the file path to the list of image paths
        image_files.append(file_path)

for filename in os.listdir(mask_dir):
    # Check if the file has a common image extension (e.g., jpg, png, etc.)
    if filename.lower().endswith('.npy'):
        # Construct the full file path by joining the folder path and filename
        file_path =  filename
        # Append the file path to the list of image paths
        mask_files.append(file_path)

pattern = r"task-(\d+)-"
for i in range(0,len(labeled_data) - 1):
    match = re.search(pattern, labeled_data[i])
    result = int(re.sub(r"task-|-", "", match.group(0)),10)
    mask[result - 1] = np.load(labeled_data[i])

mask = np.expand_dims(mask,axis=0)
print(mask)
# Iterate over each image and mask
for image_file, mask_file in zip(image_files, mask_files):
    # Load the image and mask
    image = Image.open(image_dir + '/' + image_file)
    #mask = Image.open(mask_dir + '/' + mask_file).convert('L')

    # Resize images if necessary
    image = image.resize((153, 153))
    #mask = mask.resize((153, 153))

    # Convert the image and mask to numpy arrays
    image_arr = keras.utils.img_to_array(image) / 255.0  # Normalize pixel values between 0 and 1
    #mask_arr = keras.utils.img_to_array(mask) / 255.0  # Normalize pixel values between 0 and 1

    # Expand dimensions to match input shape of the model
    image_arr = np.expand_dims(image_arr, axis=0)
    #mask_arr = np.expand_dims(mask_arr, axis=0)

# Train the model
model.fit(image_arr, mask_arr, batch_size=1, epochs=10)

# Once the model is trained, you can use it for inference on new images
# Assuming you have a new image for segmentation
new_image_file = '../cse-seminar/small_images/small_image1.png'
new_image = keras.utils.image.load_img(new_image_file, target_size=(153, 153))
new_image_arr = keras.utils.image.img_to_array(new_image) / 255.0  # Normalize pixel values between 0 and 1
new_image_arr = np.expand_dims(new_image_arr, axis=0)

# Perform inference
predicted_mask = model.predict(new_image_arr)

# Convert predicted mask to image format for visualization or further processing
predicted_mask = (predicted_mask[0] > 0.5).astype(np.uint8) * 255
predicted_mask_img = keras.utils.image.array_to_img(predicted_mask)

# Save the predicted mask image
#predicted_mask_img.save('path/to/save/predicted_mask.jpg')
plt.imshow(predicted_mask_img)


[]
Epoch 1/10


ValueError: in user code:

    File "C:\Users\alexd\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\alexd\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\alexd\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\alexd\anaconda3\lib\site-packages\keras\engine\training.py", line 995, in train_step
        self._validate_target_and_loss(y, loss)
    File "C:\Users\alexd\anaconda3\lib\site-packages\keras\engine\training.py", line 959, in _validate_target_and_loss
        raise ValueError(

    ValueError: No loss found. You may have forgotten to provide a `loss` argument in the `compile()` method.
